1. **Content-based**   
Пользователю рекомендуются объекты, похожие на те, которые этот пользователь уже употребил.   
Похожести оцениваются по признакам содержимого объектов.   
Сильная зависимость от предметной области, полезность рекомендаций ограничена.  
<p>
2. **Коллаборативная фильтрация (Collaborative Filtering)**.  
User2User / Item2Item <br>
Для рекомендации используется история оценок как самого пользователя, так и других пользователей.   
Более универсальный подход, часто дает лучший результат.   
Есть свои проблемы (например, холодный старт).   
    
**Все перечисленные методы обладают следующими недостатками**: <br>
Проблема холодного старта. <br>
Плохие предсказания для новых/нетипичных пользователей/объектов. <br>
Тривиальность рекомендаций. <br>
Ресурсоемкость вычислений. Для того, чтобы делать предсказания нам нужно держать в памяти все оценки всех пользователей. <br>
    
3. **SVD (Singular Value Decomposition)**, переводится как сингулярное разложение матрицы<br>
чтобы предсказать оценку пользователя для фильма, мы берем некоторый вектор (набор параметров) для данного пользователя и вектор для данного фильма. Их скалярное произведение и будет нужным нам предсказанием. <br>
    Но так как векторов мы не знаем, их еще нужно получить. Идея заключается в том, что у нас есть оценки пользователей, при помощи которых мы можем найти такие оптимальные параметры, при которых наша модель предсказывала бы эти оценки как можно лучше. <br>
![title](1.png) <br>
    
- градиентный спуск <br>
- Alternating Least Squares: Для каждого конкретного параметра, если мы зафиксируем все остальные, это будет как раз параболой. Т.е. минимум по одной координате мы можем точно определить. <br>
    
**Измерение качества рекомендаций** <br>
- RMSE, но у каждого пользователя свое представление о шкале оценок. Пользователи, у которых разброс оценок более широкий, будут больше влиять на значение метрики, чем другие. Ошибка в предсказании высокой оценки имеет такой же вес, что и ошибка в предсказании низкой оценки. При этом предсказать оценку 9 вместо настоящей оценки 7 страшнее, чем предсказать 4 вместо 2 (по десятибалльной шкале). Можно иметь почти идеальную метрику RMSE, но иметь очень плохое качество ранжирования, и наоборот. <br>
- метрики ранжирования (recall/precision), но нет данных про рекомендованные объекты, которые пользователь не оценивал. Оптимизировать эти метрики напрямую почти невозможно. <br>
    
**Похожесть** <br>
Похожие объекты — это объекты, похожие по своим признакам (content-based). <br>
Похожие объекты — это объекты, которые часто используют вместе («клиенты, купившие Х, также покупали А»). <br>
Похожие объекты — это рекомендации пользователю, которому понравился данный объект. <br>
Похожие объекты — это просто рекомендации, в которых данный объект выступает в качестве контекста. <br>
    
**Как учитывать дополнительную информацию?** <br>

Как учитывать не только явный (explicit), но и неявный (implicit) фидбек? (Неявного фидбека часто бывает на порядки больше.) <br>
Как учитывать контекст? (Context-aware recommendations) <br>
Как учитывать признаки объектов? (Гибридные системы) <br>
Как учитывать связи между объектами? (таксономию) <br>
Как учитывать признаки и связи пользователей? <br>
Как учитывать информацию из других источников и предметных областей? (Cross-domain recommendations) <br>

# Задание
Датасет ml-latest. <br>
Вспомнить подходы, которые мы разбирали. <br>
Выбрать понравившийся подход к гибридным системам. <br>
Написать свою. <br>

In [1]:
import pandas as pd
import numpy as np

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [4]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [5]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [6]:
tags.head(2)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


Для коллаборативной фильтрации нам нужны рейтинги. <br>
Для content-based описание товаров (в нашем случае жанры).

In [7]:
movie_x_rating = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movie_x_rating.dropna(inplace=True)
movie_x_rating.head(2)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,964982703.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,847434962.0


## Коллаборативная фильтрация

In [8]:
dataset = pd.DataFrame({
    'uid': movie_x_rating.userId, #идентификаторы пользователей
    'iid': movie_x_rating.title, #идентификаторы фильмов
    'rating': movie_x_rating.rating #взаимодействие
})

In [9]:
dataset.head(2)

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0


In [10]:
min(dataset.rating)

0.5

In [11]:
max(dataset.rating)

5.0

In [12]:
reader = Reader(rating_scale=(0.5, 5.0)) # считыватель + масштабирование данных
data = Dataset.load_from_df(dataset, reader)

In [13]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

Модель коллаборативной фильтрации со скрытыми факторами

In [14]:
%%time
algo = SVD(n_factors=60, n_epochs=20, random_state=1) # тоже фиксируем случайность чтобы не потерять самую лучшую модель
algo.fit(trainset)

CPU times: user 1.01 s, sys: 12.4 ms, total: 1.03 s
Wall time: 1.1 s


In [15]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8694


0.8693888394767368

## Content-based

In [16]:
# функция для обработки названий жанров
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [17]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [18]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)
X_train_tfidf # разрежанная спарсе матрица со значимыми числами, отличными от нуля

<9742x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22084 stored elements in Compressed Sparse Row format>

In [19]:
neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=20)

In [20]:
test = change_string("Adventure|Comedy|Fantasy|Crime")
X_tfidf2 = tfidf.transform([test])
res = neigh.kneighbors(X_tfidf2, return_distance=True)

movies.iloc[res[1][0]]

,movieId,title,genres
6774,60074,Hancock (2008),Action|Adventure|Comedy|Crime|Fantasy
9096,143559,L.A. Slasher (2015),Comedy|Crime|Fantasy
2302,3052,Dogma (1999),Adventure|Comedy|Fantasy
2608,3489,Hook (1991),Adventure|Comedy|Fantasy
9717,188833,The Man Who Killed Don Quixote (2018),Adventure|Comedy|Fantasy
5737,30810,"Life Aquatic with Steve Zissou, The (2004)",Adventure|Comedy|Fantasy
8361,109042,Knights of Badassdom (2013),Adventure|Comedy|Fantasy
6723,58972,Nim's Island (2008),Adventure|Comedy|Fantasy
7496,82854,Gulliver's Travels (2010),Adventure|Comedy|Fantasy
7865,94015,Mirror Mirror (2012),Adventure|Comedy|Fantasy


In [21]:
res[1][0]

array([6774, 9096, 2302, 2608, 9717, 5737, 8361, 6723, 7496, 7865, 3576,
       3376, 3582, 5627, 5636,  863, 3302, 2206, 6133, 5832])

In [22]:
# Создадим словарь, в котором по названию фильма можно получить его признаки
title_genres = {}

for index, row in movies.iterrows():
    title_genres[row.title] = row.genres

In [23]:
# первой должна быть та модель, которая работает лучше всего
def recommend_for_user(user_id):
    current_user_id = user_id
    user_movies = movie_x_rating[movie_x_rating.userId == current_user_id].title.unique()
    
    last_user_movie = user_movies[-1]
    
    movie_genres = title_genres[last_user_movie]
    
    movie_genres = change_string(movie_genres)

    X_tfidf2 = tfidf.transform([movie_genres])

    res = neigh.kneighbors(X_tfidf2, return_distance=True)
    
    movies_to_score = movies.iloc[res[1][0]].title.values

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)
        
    
    best_indexes = np.argsort(scores)[-10:]
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [24]:
movie_x_rating[movie_x_rating.userId == 2.0].sort_values('rating')

,movieId,title,genres,userId,rating,timestamp
97478,114060,The Drop (2014),Crime|Drama|Thriller,2.0,2.0,1.445715e+09
93998,91658,"Girl with the Dragon Tattoo, The (2011)",Drama|Thriller,2.0,2.5,1.445715e+09
8652,318,"Shawshank Redemption, The (1994)",Crime|Drama,2.0,3.0,1.445715e+09
96746,109487,Interstellar (2014),Sci-Fi|IMAX,2.0,3.0,1.445715e+09
91063,77455,Exit Through the Gift Shop (2010),Comedy|Documentary,2.0,3.0,1.445715e+09
90135,71535,Zombieland (2009),Action|Comedy|Horror,2.0,3.0,1.445715e+09
97675,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2.0,3.5,1.445715e+09
76960,8798,Collateral (2004),Action|Crime|Drama|Thriller,2.0,3.5,1.445715e+09
95272,99114,Django Unchained (2012),Action|Drama|Western,2.0,3.5,1.445715e+09
93833,91529,"Dark Knight Rises, The (2012)",Action|Adventure|Crime|IMAX,2.0,3.5,1.445715e+09


In [25]:
recommend_for_user(2.0)

Hoop Dreams (1994) 3.999142221091063
Crumb (1994) 3.9084795373511714
Wonderful, Horrible Life of Leni Riefenstahl, The (Macht der Bilder: Leni Riefenstahl, Die) (1993) 3.7192418230596913
Celluloid Closet, The (1995) 3.701008095750074
Great Day in Harlem, A (1994) 3.6735123160092886
Maya Lin: A Strong Clear Vision (1994) 3.670014314245029
Catwalk (1996) 3.62022015950832
War Room, The (1993) 3.598110834658683
Nico Icon (1995) 3.56258179027156
Microcosmos (Microcosmos: Le peuple de l'herbe) (1996) 3.5510033137094617
